In [1]:
%pip install lightning_lite
import numpy as np
import anndata as ad
import pandas as pd
import torch
from lightning_lite import seed_everything
from pytorch_lightning.callbacks import EarlyStopping  # ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch import optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

seed_everything(10, workers=True)

Note: you may need to restart the kernel to use updated packages.


c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 10


10

Loading data

In [2]:
path="C:/Users/Dell/Documents/Deep_learning_for_life_science/Projekt_zaliczeniowy/DL4LS24-IDK-main/DL4LS24-IDK-main/data/train/cell_data.h5ad"
### input Your path

anndata = ad.read_h5ad(path)
anndata.layers
anndata.X = anndata.layers['exprs'] # FIX!

df=anndata.obs
counts=anndata.layers['counts']
exprs=anndata.layers['exprs']

Appending information from exprs to dataframe for prediction task

In [3]:
list_cols_counts=['counts'+str(i) for i in range(40)]
list_cols_exprs=['exprs'+str(i) for i in range(40)]
df_counts = pd.DataFrame(data=counts, columns=list_cols_counts,index=df.index)
df_exprs = pd.DataFrame(data=exprs, columns=list_cols_exprs,index=df.index)
df_counts.shape

(236791, 40)

In [4]:
df1=df.copy()
result = pd.concat([df1, df_exprs], axis=1)
print(result.shape)
print(result.columns)

df=result.copy()

(236791, 79)
Index(['image', 'sample_id', 'ObjectNumber', 'Pos_X', 'Pos_Y', 'area',
       'major_axis_length', 'minor_axis_length', 'eccentricity', 'width_px',
       'height_px', 'acquisition_id', 'SlideId', 'Study', 'Box.Description',
       'Position', 'SampleId', 'Indication', 'BatchId', 'SubBatchId', 'ROI',
       'ROIonSlide', 'includeImage', 'flag_no_cells', 'flag_no_ROI',
       'flag_total_area', 'flag_percent_covered', 'small_cell', 'celltypes',
       'flag_tumor', 'PD1_pos', 'Ki67_pos', 'cleavedPARP_pos', 'GrzB_pos',
       'tumor_patches', 'distToCells', 'CD20_patches', 'Batch', 'cell_labels',
       'exprs0', 'exprs1', 'exprs2', 'exprs3', 'exprs4', 'exprs5', 'exprs6',
       'exprs7', 'exprs8', 'exprs9', 'exprs10', 'exprs11', 'exprs12',
       'exprs13', 'exprs14', 'exprs15', 'exprs16', 'exprs17', 'exprs18',
       'exprs19', 'exprs20', 'exprs21', 'exprs22', 'exprs23', 'exprs24',
       'exprs25', 'exprs26', 'exprs27', 'exprs28', 'exprs29', 'exprs30',
       'exprs31', '

Prepering targets and removing unimportant columns

In [5]:
cols_to_remove = ['image', 'sample_id', 'ObjectNumber', 'Pos_X', 'Pos_Y', 'width_px',
       'height_px', 'acquisition_id', 'SlideId', 'Study', 'Box.Description',
       'Position', 'SampleId', 'Indication', 'BatchId', 'SubBatchId', 'ROI',
       'ROIonSlide', 'includeImage', 'flag_no_cells', 'flag_no_ROI',
       'flag_total_area', 'flag_percent_covered', 'small_cell', 'celltypes',
       'flag_tumor', 'PD1_pos', 'Ki67_pos', 'cleavedPARP_pos', 'GrzB_pos',
       'tumor_patches', 'distToCells', 'CD20_patches', 'Batch']  # List of columns to remove

# Remove columns
df = df.drop(cols_to_remove, axis=1)


In [6]:
y=df['cell_labels']
encoder = LabelEncoder()
y_encoded1 = encoder.fit_transform(y)
df_y = pd.DataFrame(data=y_encoded1, columns=['target'],index=df.index)
# print(df_y.shape, df_y, df['cell_labels'])

In [7]:
result = pd.concat([df, df_y], axis=1)
#df=result.copy()
#df = df.drop(['cell_labels'], axis=1)
df = result.copy()
df = df.drop(['cell_labels'], axis=1)

In [10]:
y = df['target']
df.drop(['target'], axis=1)
x = df.drop(['target'], axis=1)
y

IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_1     6
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_3     7
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_5     4
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_7    11
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_8    11
                                                                    ..
IMMUcan_Batch20220908_S-220729-00002_002.tiff_2713                   7
IMMUcan_Batch20220908_S-220729-00002_002.tiff_2715                   7
IMMUcan_Batch20220908_S-220729-00002_002.tiff_2721                   3
IMMUcan_Batch20220908_S-220729-00002_002.tiff_2722                   7
IMMUcan_Batch20220908_S-220729-00002_002.tiff_2725                  11
Name: target, Length: 236791, dtype: int32

In [11]:
x

,area,major_axis_length,minor_axis_length,eccentricity,exprs0,exprs1,exprs2,exprs3,exprs4,exprs5,...,exprs30,exprs31,exprs32,exprs33,exprs34,exprs35,exprs36,exprs37,exprs38,exprs39
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_1,13.0,6.094800,2.780135,0.889904,0.000000,1.819988,0.502650,1.265410,0.515007,3.331261,...,0.792218,0.567238,1.651610,2.556218,0.777378,0.000000,3.581082,0.424273,4.761130,5.301351
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_3,56.0,21.520654,3.368407,0.987675,0.135920,2.393087,1.136961,0.505529,0.605836,0.710857,...,0.530987,0.603468,1.358368,1.700181,1.884778,0.068872,0.744634,0.118620,5.029241,5.530764
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_5,12.0,5.294329,2.862220,0.841267,0.165905,2.244917,1.396334,1.744382,0.458541,4.006500,...,0.449097,0.812795,1.949510,3.465199,1.743507,0.403610,2.244398,0.015141,4.418390,4.815923
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_7,12.0,5.294329,2.862220,0.841267,0.165905,1.144778,0.563216,1.166010,0.513735,2.034705,...,1.866916,1.228290,2.287524,2.491710,2.268924,0.196199,0.946492,0.233095,3.736089,4.368012
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_8,29.0,9.216670,4.112503,0.894932,0.137497,2.210970,0.892371,0.497646,0.412840,1.499184,...,0.806346,0.360098,1.445085,2.332546,2.387292,0.061568,1.632904,0.372278,4.607359,5.152041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IMMUcan_Batch20220908_S-220729-00002_002.tiff_2713,31.0,6.857501,5.700162,0.555928,0.223930,5.665851,2.734667,2.338794,0.853400,1.660905,...,0.402421,0.151767,0.820879,1.591254,1.382112,0.813415,0.531336,0.034725,4.334035,4.838697
IMMUcan_Batch20220908_S-220729-00002_002.tiff_2715,20.0,6.484816,3.840203,0.805803,0.149443,3.908886,2.751400,0.702922,0.539905,0.435384,...,0.149241,0.069210,0.256412,1.246646,1.200277,0.230447,0.203544,0.000000,4.633061,5.182098
IMMUcan_Batch20220908_S-220729-00002_002.tiff_2721,27.0,10.732613,3.134663,0.956397,0.110884,3.520289,0.310834,2.590231,2.336469,3.652604,...,0.667975,0.434373,1.868607,2.931921,1.427727,0.994726,0.224414,0.102515,4.303070,4.826098
IMMUcan_Batch20220908_S-220729-00002_002.tiff_2722,33.0,12.864691,3.228974,0.967988,0.150941,4.214958,2.651646,0.807826,0.403674,1.226701,...,0.256219,0.096142,0.244681,1.495089,0.792030,0.047485,0.090598,0.083975,4.822456,5.361662


In [19]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

(47359,)

In [47]:
def create_and_evaluate_model(X_train, X_test, y_train, y_test):
    # Create and fit the logistic regression model
    model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy across all classes: ", accuracy)
    calculate_accuracy_across_clases(model, X_test, y_test)
    return model

def one_hot_encoding(array, num_classes = 14):
    # Create a zero tensor with the desired shape
    tensor = torch.tensor(array)
    one_hot = torch.zeros(tensor.size(0), num_classes)
    # Use scatter_ to fill the one-hot tensor
    one_hot.scatter_(1, tensor.unsqueeze(1).long(), 1)
    return one_hot.numpy()

def calculate_accuracy_across_clases(model, x_test, y_test):
    y_pred = model.predict(x_test)
    pred_one_hot = one_hot_encoding(y_pred)
    labels_one_hot = one_hot_encoding(y_test)
    corect = pred_one_hot * labels_one_hot
    total_number_of_examples_per_class = np.sum(labels_one_hot, axis = 0)
    total_number_of_corect_examples_per_class = np.sum(corect, axis = 0)
    print(f'Total no of examples:, total no of corect ansvers:, accuracy per class: \n {total_number_of_examples_per_class} \n {total_number_of_corect_examples_per_class} \n {np.round(total_number_of_corect_examples_per_class /total_number_of_examples_per_class, 4)}')


In [48]:
model = create_and_evaluate_model(X_train, X_test, y_train, y_test)

c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Dell\AppData\Local\Temp\ipykernel_9432\2390085703.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tensor = torch.tensor(array)


Accuracy across all classes:  0.9333600793935682
Total no of examples:, total no of corect ansvers:, accuracy per class: 
 [  981.  1385.  2512.  4220.   917.   742.  3160.  4434.   219.  1457.
  1149. 24257.   326.  1600.] 
 [  891.  1248.  2222.  3964.   559.   388.  2677.  4219.   159.  1390.
  1076. 23655.   282.  1473.] 
 [0.9083 0.9011 0.8846 0.9393 0.6096 0.5229 0.8472 0.9515 0.726  0.954
 0.9365 0.9752 0.865  0.9206]
